# Probes across all layers

We score SAEs by their ability to "recover" supervised concepts from the residual stream? To which degree are those concepts detectable by linear probes at all?

In [1]:
# Imports

In [2]:
# Load model

In [3]:
# Load dataset

In [6]:
# Define activation caching

In [5]:
# Define Linear probes, and training

In [ ]:
# Interate over all layers